In [2]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import pickle

In [3]:
def load_dataset(folder_path: str):

    data_frames = []
    for csv in Path(folder_path).glob("*.csv"):
        df = pd.read_csv(csv)
        data_frames.append(df)

    combined_df = pd.concat(data_frames, ignore_index=True)

    combined_df = combined_df[(combined_df["aa"].notna()) & (combined_df["dssp"].notna())]
    combined_df['dssp3'] = combined_df['dssp'].map({
        'H': 'H',
        'G': 'H',
        'I': 'H',
        'P': 'H',
        'B': 'B',
        'E': 'B',
        '.': '.',
        'T': '.',
        'S': '.'
    })

    return combined_df

In [4]:
df_alpha = load_dataset("data/ca-features")
df_beta = load_dataset("data/cb-features")
df_com = load_dataset("data/com-features")

dfs = {
    'alpha': df_alpha,
    'beta': df_beta,
    'com': df_com,
}

In [5]:
dist_cols = [col for col in df_alpha.columns if col.startswith('dist_')]
angle_cols = [col for col in df_alpha.columns if col.startswith('angle_') or col.startswith('dihedral_')]
neighbor_cols = [col for col in df_alpha.columns if col.startswith('neighbor_')]

In [ ]:
X = df_alpha[angle_cols + dist_cols + neighbor_cols +["aa"]]
y = df_alpha["dssp3"]

In [ ]:
X_test, X_train, y_test, y_train = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
def preproc_random_forest(X, y):
    aa_enc = OneHotEncoder(sparse_output=False)
    aa_encoded = aa_enc.fit(X[["aa"]]).transform(X[["aa"]])

    aa_encoded_df = pd.DataFrame(aa_encoded, columns=aa_enc.get_feature_names_out(["aa"]))
    X_processed = pd.concat([X.reset_index(drop=True).drop(columns=["aa"]), aa_encoded_df.reset_index(drop=True)], axis=1)

    return X_processed, y

def train_random_forest(X_train, y_train):
    model = RandomForestClassifier(random_state=42)

    X_train, y_train = preproc_random_forest(X_train, y_train)

    params = {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, 50, 100],
    }

    cv = GridSearchCV(
        estimator=model,
        param_grid=params,
        scoring='accuracy',
        cv=StratifiedKFold(n_splits=5),
        return_train_score=True,
        n_jobs=-1,
        verbose=False
    )

    cv.fit(X_train, y_train)

    return cv

def test_random_forest(X_test, y_test, model):
    X_test, y_test = preproc_random_forest(X_test, y_test)

    y_pred = model.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix - Random Forest')

    return classification_report(y_test, y_pred)

In [7]:
xgboost_label_mapping = {'H': 0, 'B': 1, '.': 2}

def preproc_xgboost(X, y):

    X_processed = X.copy()
    X_processed['aa'] = X_processed['aa'].astype('category')

    y_processed = y.map(xgboost_label_mapping)

    return X_processed, y_processed

def train_xgboost(X_train, y_train):

    X_train, y_train = preproc_xgboost(X_train, y_train)

    model = XGBClassifier(
        objective='multi:softmax',
        num_class=3,
        enable_categorical=True,
    )

    params = {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, 50, 100],
        'learning_rate': [0.1, 0.2, 0.3]
    }

    cv = GridSearchCV(
        estimator=model,
        param_grid=params,
        scoring='accuracy',
        cv=StratifiedKFold(n_splits=5),
        return_train_score=True,
        n_jobs=-1,
        verbose=False
    )

    cv.fit(X_train, y_train)

    return cv

def test_xgboost(X_test, y_test, model):
    X_test, y_test = preproc_xgboost(X_test, y_test)

    y_pred = model.predict(X_test)

    keys = list(xgboost_label_mapping.keys())

    cm = confusion_matrix(
        y_test,
        y_pred
    )
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=keys, yticklabels=keys)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix - XGBoost')

    return classification_report(y_test, y_pred)

In [8]:
def preproc_catboost(X, y):

    X_processed = X.copy()
    X_processed['aa'] = X_processed['aa'].astype('category')

    y_processed = y.astype('category')

    return X_processed, y_processed

def train_catboost(X_train, y_train):

    X_train, y_train = preproc_catboost(X_train, y_train)

    model = CatBoostClassifier(
        loss_function='MultiClass',
        cat_features=['aa'],
    )
    model.grid_search(
        param_grid={
            'iterations': [100, 200],
            'depth': [4, 6, 10],
            'learning_rate': [0.01, 0.1, 0.2]
        },
        X=X_train,
        y=y_train,
        cv=StratifiedKFold(n_splits=5),
        verbose=False
    )

    return model

def test_catboost(X_test, y_test, model):
    X_test, y_test = preproc_catboost(X_test, y_test)

    y_pred = model.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix - CatBoost')

    return classification_report(y_test, y_pred)

In [10]:
train_fns = {
    'random_forest': train_random_forest,
    'xgb': train_xgboost,
    'catboost': train_catboost
}

for data_name, df in dfs.items():

    X = df[angle_cols + dist_cols + neighbor_cols +["aa"]]
    y = df["dssp3"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    for model_name, train_fn in train_fns.items():

        model_key = f"{model_name}_{data_name}"
        outname = f"models/{model_key}_model.pkl"
        model = train_fn(X_train, y_train)

        pickle.dump(model, open(outname, "wb"))

/Users/aerlaut/Projects/corssa/.venv/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/aerlaut/Projects/corssa/.venv/lib/python3.13/site-packages/sklearn/model_selection/_split.py:885: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


0:	learn: 1.0887855	test: 1.0887021	best: 1.0887021 (0)	total: 63.1ms	remaining: 6.24s
1:	learn: 1.0801060	test: 1.0801380	best: 1.0801380 (1)	total: 69.6ms	remaining: 3.41s
2:	learn: 1.0706316	test: 1.0705768	best: 1.0705768 (2)	total: 75.2ms	remaining: 2.43s
3:	learn: 1.0616333	test: 1.0613150	best: 1.0613150 (3)	total: 82.4ms	remaining: 1.98s
4:	learn: 1.0532209	test: 1.0528681	best: 1.0528681 (4)	total: 88.2ms	remaining: 1.68s
5:	learn: 1.0449577	test: 1.0447395	best: 1.0447395 (5)	total: 90.4ms	remaining: 1.42s
6:	learn: 1.0365130	test: 1.0364030	best: 1.0364030 (6)	total: 94.9ms	remaining: 1.26s
7:	learn: 1.0280918	test: 1.0278724	best: 1.0278724 (7)	total: 97.4ms	remaining: 1.12s
8:	learn: 1.0201606	test: 1.0197743	best: 1.0197743 (8)	total: 99.4ms	remaining: 1s
9:	learn: 1.0120971	test: 1.0117264	best: 1.0117264 (9)	total: 102ms	remaining: 918ms
10:	learn: 1.0042701	test: 1.0037741	best: 1.0037741 (10)	total: 104ms	remaining: 842ms
11:	learn: 0.9967028	test: 0.9961917	best: 0.9

/Users/aerlaut/Projects/corssa/.venv/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/aerlaut/Projects/corssa/.venv/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/aerlaut/Projects/corssa/.venv/lib/python3.13/site-packages/sklearn/model_selection/_split.py:885: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


0:	learn: 1.0895913	test: 1.0900265	best: 1.0900265 (0)	total: 3.81ms	remaining: 377ms
1:	learn: 1.0808905	test: 1.0816297	best: 1.0816297 (1)	total: 5.97ms	remaining: 293ms
2:	learn: 1.0724802	test: 1.0735481	best: 1.0735481 (2)	total: 9.41ms	remaining: 304ms
3:	learn: 1.0638920	test: 1.0651910	best: 1.0651910 (3)	total: 12ms	remaining: 288ms
4:	learn: 1.0555854	test: 1.0569696	best: 1.0569696 (4)	total: 14.1ms	remaining: 269ms
5:	learn: 1.0474570	test: 1.0491692	best: 1.0491692 (5)	total: 16.5ms	remaining: 259ms
6:	learn: 1.0394259	test: 1.0413998	best: 1.0413998 (6)	total: 18.7ms	remaining: 248ms
7:	learn: 1.0321192	test: 1.0344378	best: 1.0344378 (7)	total: 20.8ms	remaining: 239ms
8:	learn: 1.0244995	test: 1.0270535	best: 1.0270535 (8)	total: 22.8ms	remaining: 230ms
9:	learn: 1.0172377	test: 1.0199591	best: 1.0199591 (9)	total: 26.3ms	remaining: 237ms
10:	learn: 1.0102805	test: 1.0129065	best: 1.0129065 (10)	total: 28.3ms	remaining: 229ms
11:	learn: 1.0032171	test: 1.0061167	best: 

/Users/aerlaut/Projects/corssa/.venv/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/aerlaut/Projects/corssa/.venv/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/aerlaut/Projects/corssa/.venv/lib/python3.13/site-packages/sklearn/model_selection/_split.py:885: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


0:	learn: 1.0903632	test: 1.0908003	best: 1.0908003 (0)	total: 2.72ms	remaining: 269ms
1:	learn: 1.0822853	test: 1.0829737	best: 1.0829737 (1)	total: 4.86ms	remaining: 238ms
2:	learn: 1.0743852	test: 1.0753579	best: 1.0753579 (2)	total: 7.33ms	remaining: 237ms
3:	learn: 1.0664795	test: 1.0678110	best: 1.0678110 (3)	total: 9.7ms	remaining: 233ms
4:	learn: 1.0589757	test: 1.0605257	best: 1.0605257 (4)	total: 12.1ms	remaining: 231ms
5:	learn: 1.0509803	test: 1.0529154	best: 1.0529154 (5)	total: 15.9ms	remaining: 249ms
6:	learn: 1.0440343	test: 1.0461462	best: 1.0461462 (6)	total: 18.1ms	remaining: 240ms
7:	learn: 1.0368128	test: 1.0392642	best: 1.0392642 (7)	total: 20ms	remaining: 231ms
8:	learn: 1.0294854	test: 1.0321759	best: 1.0321759 (8)	total: 22.5ms	remaining: 227ms
9:	learn: 1.0225832	test: 1.0256817	best: 1.0256817 (9)	total: 24.6ms	remaining: 222ms
10:	learn: 1.0157871	test: 1.0191118	best: 1.0191118 (10)	total: 26.7ms	remaining: 216ms
11:	learn: 1.0087730	test: 1.0123010	best: 1

In [11]:
# Test models on other data
test_fns = {
    'random_forest': test_random_forest,
    'xgb': test_xgboost,
    'catboost': test_catboost
}

for model_file in Path("models").glob("*.pkl"):

    model = pickle.load(open(model_file, "rb"))

    for data_name, df in dfs.items():

        X = df[angle_cols + dist_cols + neighbor_cols +["aa"]]
        y = df["dssp3"]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        model_name = "_".join(model_file.stem.split("_")[:-2])
        report = test_fns[model_name](X_test, y_test, model)

        eval_name = f"{model_file.stem.replace("_model", "")}_eval_{data_name}"

        report_name = f"reports/{eval_name}_report.txt"

        with open(report_name, "w") as f:
            f.write(report)

        plt.savefig(f"reports/{eval_name}.png")
        plt.clf()

/var/folders/19/ytl4dbx901j_tzvz8shzy1740000gn/T/ipykernel_79183/3881670285.py:39: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(8, 6))


<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

...

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>